In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('get_follower_data.csv')

In [3]:
df.head()

,Unnamed: 0,hours,mins,secs,text
0,0,16,30,36,How to UNPIVOT multiple columns into tidy pair...
1,1,16,0,4,"Shortest Path Algorithms, Part 2: Floyd�Warsha..."
2,2,15,8,1,Code to run SQL queries 10 times faster than B...
3,3,14,41,43,New to Data Visualization? Start with New York...
4,4,4,57,47,I wrote a Colab notebook that introduces diffe...


In [4]:
df['text'] = df['text'].apply(lambda x: x[0:-1].replace('\n\n', ' '))

In [5]:
import re

def clear_emoji(text):
    emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [6]:
df['text']=df['text'].apply(clear_emoji)

In [7]:
import numpy as np
import gensim

In [8]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore



In [9]:
import spacy

In [10]:

from spacy.tokenizer import Tokenizer

In [11]:
nlp = spacy.load("en_core_web_lg")

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [12]:

STOP_WORDS = nlp.Defaults.stop_words.union(['&amp;',"don't","i'm","i've","it’s","&gt;","i’m",'de','la','que','un','=','con','y','like',"you're",'en','el','thank','+',"don’t","it's","⦁","we're",'d…','los','fucking','para','del',"here's","can't","aren’t"])


In [13]:
tokens = []

""" Update those tokens w/o stopwords"""
for doc in tokenizer.pipe(df['text'], batch_size=500):
    
    doc_tokens = []
    
    for token in doc:
        if (token.text.lower() not in STOP_WORDS) & (token.is_punct == False) & (token.is_space == False):
            doc_tokens.append(token.text.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [14]:
id2word = corpora.Dictionary(df['tokens'])

In [15]:
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [16]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=8
                  )

In [17]:
lda.print_topics()

[(0,
  '0.003*"new" + 0.003*"good" + 0.002*"data" + 0.002*"learn" + 0.002*"learning" + 0.002*"going" + 0.002*"morning" + 0.002*"got" + 0.002*"people" + 0.002*"time"'),
 (1,
  '0.005*"data" + 0.003*"today" + 0.002*"new" + 0.002*"years" + 0.002*"going" + 0.002*"sure" + 0.002*"lot" + 0.002*"lo" + 0.002*"world" + 0.002*"t…"'),
 (2,
  '0.003*"people" + 0.003*"new" + 0.002*"data" + 0.002*"free" + 0.002*"great" + 0.002*"today" + 0.002*"machine" + 0.002*"las" + 0.002*"day" + 0.002*"want"'),
 (3,
  '0.003*"need" + 0.003*"data" + 0.002*"#ai" + 0.002*"new" + 0.002*"use" + 0.002*"covid-19" + 0.002*"order" + 0.002*"work" + 0.002*"going" + 0.002*"people"'),
 (4,
  '0.002*"day" + 0.002*"place" + 0.002*"world" + 0.002*"r" + 0.002*"learning" + 0.001*"covid-19" + 0.001*"research" + 0.001*"let\'s" + 0.001*"know" + 0.001*"time"'),
 (5,
  '0.003*"love" + 0.003*"ibm" + 0.003*"|" + 0.003*"new" + 0.003*"good" + 0.002*"people" + 0.002*"social" + 0.002*"favorite" + 0.002*"excited" + 0.002*"things"'),
 (6,
  '0.

In [18]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [19]:

topics = [' '.join(t[0:5]) for t in words]

In [20]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n\n")

------ Topic 0 ------
new good data learn learning

------ Topic 1 ------
data today new years going

------ Topic 2 ------
people new data free great

------ Topic 3 ------
need data #ai new use

------ Topic 4 ------
day place world r learning

------ Topic 5 ------
love ibm | new good

------ Topic 6 ------
new need covid-19 want learning

------ Topic 7 ------
new data use want great

------ Topic 8 ------
new work data people thanks

------ Topic 9 ------
people think know new time

------ Topic 10 ------
love code people day heard

------ Topic 11 ------
best great way i’ve new

------ Topic 12 ------
game working days people coronavirus

------ Topic 13 ------
time learning #ai data people

------ Topic 14 ------
people data know time ⠀

